In [1]:
#Implementing SCD Type 2

In [2]:
import pandas as pd
import sqlalchemy as sal
engine=sal.create_engine('mssql://DESKTOP-N6DEIMB\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()

In [3]:
#implementing scd type2
def extract():
    df_product=pd.read_csv('products1.txt')
    df_product_db=pd.read_sql_query("select * from product_scd2 where expiry_date='9999-12-31'",con=conn)
    return df_product,df_product_db

def transform(df_product,df_product_db):
    df_merge=pd.merge(df_product,df_product_db,how='inner',on='product_id')
    df_update_final=df_merge.iloc[:,0:5]
    df_update_final.columns=df_product_db.columns[1:]
    return df_update_final

def staging(df_update_final):
    b=pd.read_sql_query('truncate table product_scd2_stg select 1',con=conn)
    df_update_final.to_sql('product_scd2_stg',con=conn,index=False,if_exists='append')
    conn.commit()

def history(df_update_final):
    query=sal.text("update product_scd2 set expiry_date = cast(dateadd(day,-1,product_scd2_stg.enrolled_date) as date) from product_scd2_stg where product_scd2.product_id=product_scd2_stg.product_id and product_scd2.expiry_date=(select max(expiry_date) from product_scd2)")
    p = conn.execute(query)
    conn.commit()

def insert(df_product):
    df_product.to_sql('product_scd2',con=conn,if_exists='append',index=False)
    conn.commit()

In [13]:
df_product,df_product_db=extract()

In [14]:
df_update_final=transform(df_product,df_product_db)

In [15]:
staging(df_update_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [16]:
history(df_update_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [17]:
insert(df_product)

AttributeError: 'Connection' object has no attribute 'commit'